# Upload 
> Web Interface to Flash ESP32's using Web Serial.

- prettify: true

<img src='https://charleskarpati.com/images/covers/upload1.webp'>

In [1]:
%%html 
    <style> 
        body { font-family: Arial, sans-serif; margin: 20px; }
        .container { max-width: 800px; margin: 0 auto; }
        #output { height: 300px; border: 1px solid #ccc; padding: 10px; overflow-y: auto; background-color: #f5f5f5; font-family: monospace; }
        button { background-color: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer; margin-top: 10px; }
        button:hover { background-color: #45a049; }
        button:disabled { background-color: #cccccc; cursor: not-allowed; }
        select, textarea { width: 100%; padding: 8px; margin: 10px 0; box-sizing: border-box; }
        textarea { height: 400px; font-family: monospace; }
        .preset-section { margin: 10px 0; border: 1px solid #ddd; padding: 10px; background-color: #f9f9f9; }
        .progress-bar {
            width: 100%;
            background-color: #ddd;
            border-radius: 4px;
            margin-top: 10px;
            display: none;
        }
        .progress {
            height: 20px;
            background-color: #4CAF50;
            border-radius: 4px;
            text-align: center;
            color: white;
            line-height: 20px;
            width: 0%;
        }
        textarea{
            height: 600px;
        }
    </style>
</head> 
<div class="container"> 
    <div>
        <button id="connectBtn">Connect to Device</button>
        <button id="disconnectBtn" disabled>Disconnect</button>
        <span id="connectionStatus">Not connected</span>
    </div>
    
    <div class="preset-section">
        <h2>Preset Examples:</h2>
        <select id="presetSelect">
            <option value="">-- Select an example --</option>
            <option value="blinkCounter">Blink with Counter</option>
            <option value="blinkMonitor">Blink with Monitor</option> 
            <option value="mainProject">Lightstrip Code</option> 
        </select>
        <button id="loadPresetBtn">Load Example</button>
    </div>
    
    <div>
        <h2>Code:</h2>
        <textarea id="codeArea" placeholder="Enter your Arduino code here"></textarea>
        <button id="uploadBtn" disabled>Upload Code</button>
        <button id="loadFileBtn">Load from File</button>
        <input type="file" id="fileInput" style="display: none;">
        <div class="progress-bar" id="uploadProgress">
            <div class="progress" id="progressBar">0%</div>
        </div>
    </div>
    
    <div>
        <h2>Output:</h2>
        <div id="output"></div>
    </div>
</div>


<script src="web.js" type="module"></script> 

<details>
<summary>

## In Python:

</summary>

<details>
<summary>

### Optional - Setup Localhost on Colabs

</summary>

Run this in the local terminal:

jupyter notebook \ \
  --NotebookApp.allow_origin='https://colab.research.google.com' \ \
  --port=8888 \ \
  --NotebookApp.port_retries=0 \ \

'Then hit the CONNECT dropdown on the top right of this screen to Connect to the Localhost 

</details>
<details open>
<summary>

### Upload Function

</summary>

This is the upload script:

In [2]:
# Add your user to the dialout group
#sudo usermod -a -G dialout $USER
# Verify the group was added
#groups $USER
# Check the permissions on the serial port
#ls -l /dev/ttyUSB0

In [3]:
#show_collapsed
def check_system_status():
    import os
    import subprocess
    import pwd
    
    def run_command(cmd):
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            return result.stdout.strip()
        except Exception as e:
            return f"Error: {str(e)}"
    
    # Get current user
    current_user = os.getenv('USER')
    print(f"=== Current User ===")
    print(f"Username: {current_user}")
    
    # Check user groups
    # print(f"\n=== User Groups ===")
    # print(run_command('groups $USER'))
    
    # Check serial port permissions
    print(f"\n=== Serial Port Status ===")
    print(run_command('ls -l /dev/ttyUSB0'))
    
    # Check udev rules
    print(f"\n=== UDEV Rules ===")
    print(run_command('ls -l /etc/udev/rules.d/*usb*.rules'))
    
    # Check USB devices
    print(f"\n=== USB Devices ===")
    print(run_command('lsusb | grep -i "cp2102"'))
    
    # Check dmesg for USB device
    print(f"\n=== DMesg USB Logs ===")
    print(run_command('dmesg | grep -i "cp2102" | tail -n 5'))

# Run the check
check_system_status()

=== Current User ===
Username: carlos

=== Serial Port Status ===
crw-rw-rw- 1 root dialout 188, 0 Jun 18 22:03 /dev/ttyUSB0

=== UDEV Rules ===


=== USB Devices ===


=== DMesg USB Logs ===



In [4]:
!lsusb

Bus 004 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 003 Device 002: ID 0408:4039 Quanta Computer, Inc. ACER FHD User Facing
Bus 003 Device 004: ID 10c4:ea60 Silicon Labs CP210x UART Bridge
Bus 003 Device 003: ID 8087:0033 Intel Corp. 
Bus 003 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub
Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub
Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub


In [5]:
!pip install pyserial

In [15]:
import subprocess
import time
import os
import platform
import serial.tools.list_ports
from IPython.display import clear_output

 
def get_serial_port():
    ports = list(serial.tools.list_ports.comports())
    if not ports:
        print("No serial ports found")
        return None
        
    print("Available ports:")
    for i, (port, desc, hwid) in enumerate(ports):
        print(f"{i}: {port} - {desc} [{hwid}]")
    
    if len(ports) == 1:
        return ports[0][0]
    
    try:
        choice = int(input(f"Select port (0-{len(ports)-1}): "))
        return ports[choice][0]
    except (ValueError, IndexError):
        print("Invalid selection")
        return None

def upload_code(code):
    # Get available port first
    port = get_serial_port()
    if not port:
        return False

    # Save the code to a file
    with open('./src/main.cpp', 'w') as f:
        f.write(code)

    # Get the appropriate platformio command
    pio_cmd = os.path.expanduser('~/.platformio/penv/bin/platformio')

    if not os.path.exists(pio_cmd):
        print(f"Error: PlatformIO not found at {pio_cmd}")
        print("Please ensure PlatformIO is installed and in the default location")
        return False

    try:
        # Compile and upload the code
        cmd = f'"{pio_cmd}" run --target upload --upload-port {port}'
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, universal_newlines=True)
        timeout = time.time() + 60*2  # 2 minutes from now

        # Process output
        while True:
            if time.time() > timeout:
                print("Timeout reached")
                process.kill()
                return False

            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            print(output.strip())

        # Get the output and errors
        output, error = process.communicate()
        if error:
            print("Error:", error)
            return False

        return True

    except Exception as e:
        print(f"Error during upload: {str(e)}")
        return False

def read_and_upload_code(file_path):
    with open(file_path, 'r') as f:
        code = f.read()
        return upload_code(code)

In [16]:
import serial.tools.list_ports

def list_available_ports():
    ports = serial.tools.list_ports.comports()
    available_ports = []
    for port, desc, hwid in sorted(ports):
        print(f"{port}: {desc} [{hwid}]")
        available_ports.append(port)
    return available_ports

print("Available ports:")
available_ports = list_available_ports()

Available ports:
/dev/ttyUSB1: CP2102 USB to UART Bridge Controller - CP2102 USB to UART Bridge Controller [USB VID:PID=10C4:EA60 SER=0001 LOCATION=3-4]


</details>
<details open>
<summary>

### Uploading

</summary>

This is the upload fn in use:

In [17]:
import os
directory = os.path.join(os.getcwd(), 'samples')
files = os.listdir(directory)
for file in files:
    print(file)

# file = 'a_FastLedFTT.cpp'
# file = 'analogSoundSampleSketch.cpp'
# file = 'gpt4_v4.cpp' 
# file = 'blink.cpp'

_analogAudio.cpp
_fft_v2.cpp
blink_monitor.cpp
a_mic_2_envelope_analyzed.cpp
a_FastLedFTT.cpp
_digitalAudio.cpp
a_theatersound.cpp
_fft_v1.cpp
blink_serial.cpp
a_sound2light3.cpp


In [18]:
directory = os.path.join(os.getcwd(), 'lightstrip') 
file = 'main.cpp'
fin = directory+'/'+file

In [19]:
directory

'/home/carlos/Documents/GitHub/lights/lightstrip'

In [20]:
fin

'/home/carlos/Documents/GitHub/lights/lightstrip/main.cpp'

fin

In [21]:
directoryy = os.path.join(os.getcwd(), 'samples') 
filee = 'blink_serial.cpp'
finn = directoryy+'/'+filee

In [24]:
read_and_upload_code(fin)

Available ports:
0: /dev/ttyUSB1 - CP2102 USB to UART Bridge Controller - CP2102 USB to UART Bridge Controller [USB VID:PID=10C4:EA60 SER=0001 LOCATION=3-4]
Processing esp32dev (platform: espressif32; board: esp32dev; framework: arduino)
--------------------------------------------------------------------------------
Verbose mode can be enabled via `-v, --verbose` option
CONFIGURATION: https://docs.platformio.org/page/boards/espressif32/esp32dev.html
PLATFORM: Espressif 32 (6.10.0) > Espressif ESP32 Dev Module
HARDWARE: ESP32 240MHz, 320KB RAM, 4MB Flash
DEBUG: Current (cmsis-dap) External (cmsis-dap, esp-bridge, esp-prog, iot-bus-jtag, jlink, minimodule, olimex-arm-usb-ocd, olimex-arm-usb-ocd-h, olimex-arm-usb-tiny-h, olimex-jtag-tiny, tumpa)
PACKAGES:
- framework-arduinoespressif32 @ 3.20017.241212+sha.dcc1105b
- tool-esptoolpy @ 1.40501.0 (4.5.1)
- tool-mkfatfs @ 2.0.1
- tool-mklittlefs @ 1.203.210628 (2.3)
- tool-mkspiffs @ 2.230.0 (2.30)
- toolchain-xtensa-esp32 @ 8.4.0+2021r2-pat

False

</details>
<details open>
<summary>

### Serial Logging

</summary>

This is a simple example to read the serial output

In [16]:
import serial
import time
from IPython.display import clear_output

def setup_serial_monitor(port=None, baudrate=115200):
    if port is None:
        port = get_serial_port()
    if not port:
        return None
        
    try:
        ser = serial.Serial(port, baudrate, timeout=1)
        ser.flushInput()
        return ser
    except serial.SerialException as e:
        print(f"Error opening port {port}: {e}")
        return None

ser = setup_serial_monitor()
if ser:
    timeout = 30
    start_time = time.time()  # Record the start time

    try:
        while True:
            current_time = time.time()
            if current_time - start_time > timeout: 
                print("Time elapsed. Exiting...")
                break

            inpt = ser.readline()
            if inpt:
                print(inpt.decode('utf-8').strip())
    except KeyboardInterrupt:
        print("Keyboard Interrupt. Exiting...")
    finally: 
        ser.close()  # Ensure the serial connection is closed
        clear_output()


SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

</details>
<details open>
<summary>

### Serial Plotting

</summary>

This is a more complex example

In [ ]:
#collapse_input #hide_output
monitor = """
import serial
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import csv
import time

ser = serial.Serial()
ser.baudrate = 115200
ser.port = 'COM11'
ser.open()
ser.flushInput()

# Create an empty list to store the data
data = [0] * 100

# Create the plot
plt.ion() # Turn on interactive mode
fig, ax = plt.subplots()
line, = ax.plot(data)
ax.set_title("Random Data")
ax.set_xlabel("Data Point")
ax.set_ylabel("Value")

try:
    while True:
        # Read a line of data from the serial port
        inpt = ser.readline().decode('utf-8').strip()

        # Parse the data and append it to the list
        try:
            value = int(inpt)
            data.pop(0)  # Remove the first value from the list
            data.append(value)  # Append the new value to the end of the list
        except IndexError:
            pass

        # Update the plot
        line.set_ydata(data)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw()
        fig.canvas.flush_events()

        # Display the plot
        clear_output(wait=True)
        display(fig)

        # save the output 
        with open("test_data.csv","a", newline="") as f:
            writer = csv.writer(f,delimiter=",")
            writer.writerow([time.time(),value])

except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally:
    ser.close()
    clear_output()
"""

In [ ]:
import os

old = """
def list_ino_files(root_dir):
    ino_files = []
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            ino_file = os.path.join(folder_path, folder + '.ino')
            if os.path.isfile(ino_file):
                ino_files.append(ino_file)
    return ino_files

def print_ino_file_contents(file_index, ino_files):
    if 0 <= file_index < len(ino_files):
        with open(ino_files[file_index], 'r') as file:
            contents = file.read()
            print(ino_files[file_index])
            print(contents)
    else:
        print("Invalid index.")

# Change this path to your root directory
root_dir = r"/home/carlos/Documents/GitHub/lights/upload.ipynb\Arduino\scripts"

# Listing and printing files with indexes
ino_files = list_ino_files(root_dir)
for index, file in enumerate(ino_files):
    relative_path = os.path.relpath(file, root_dir)
    print(f"Index: {index}, File: {relative_path}")"""

In [ ]:
# Example usage
# print_ino_file_contents(28, ino_files)  # Replace 0 with your chosen index

platformio run -target upload

</details>
<details open>
<summary>

</details>

</details>